In [1]:
%reset
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from functions import loader, splt, fitter, stats, grouper
import matplotlib.pyplot as plt
from uncertainties import unumpy as unp
from uncertainties import ufloat

savedir = "G01/"

In [3]:
all_data = loader.load_data( "../Plans20221106TheHugescan_20221114_v2" )
data = loader.remove_duplicates( all_data,
    lambda d: ( d['dep_polymers'], d['side'], d['defects_frac'], d['gamma'] ),
    lambda d: d['runned_replicas']
)
len( data )

29

In [4]:
filtered_data = data

In [5]:
loader.export( filtered_data, savedir + "data.json" )

29  rows exported to  G01/data.json


In [11]:
from functions import CEfitter

to_use = lambda d: d['occupation_std']

grouped_data = grouper.extract_xy( filtered_data, ['dep_polymers', 'defects_frac', 'gamma'], lambda d: d['side'], to_use )

grouped_data = grouped_data[ np.vectorize( lambda g : len( g['x'] ) > 2 )( grouped_data ) ]

# splt.init( len( grouped_data ) / 4 + 1, 4 )
for i_g, g in enumerate( grouped_data ):
    # splt.next()
    # print( i_g, end=': ')
    g['B'], logx, logyB = CEfitter.estimate_B_errony( g['x'] , g['y'] )
    # plt.title( i_g )
    # g['oneovernu'], g['somethingrelatedtoA'] = CEfitter.estimate_oneovernu( logx, logyB )
    g['oneovernu_b'], g['somethingrelatedtoA_b'] = CEfitter.estimate_oneovernu_bis( logx, logyB, 0.013 )

print( len( grouped_data ) )

0


In [8]:
splt.oneitem_iterate_errorbar(
    len( grouped_data ),
    lambda d: grouped_data[d]['dep_polymers'],
    lambda d: grouped_data[d]['defects_frac'],
    lambda d: grouped_data[d]['gamma'],
    lambda d: [
        grouped_data[d]['oneovernu_b']
    ],
    lambda it, l: [
        { 'label': fr"$q = {l}$", 'ls': '-', 'marker': 's' },
    ],
    lambda it: (
        plt.title( fr"{it}" ),
        plt.xlabel( fr"$\gamma$" ),
        plt.ylabel( fr"$1/\nu$" ),
        plt.ylim( 0.8, 1.15 ),
        splt.hline( 1 )
    ),
    pvals_ys = [ 1.13, 1.11, 1.09 ],
    pvals_exp = 1
)

plt.tight_layout()
plt.savefig( savedir + '/oneovernu.pdf' )
# plt.savefig( savedir + '/oneovernu.png' )

<Figure size 432x0 with 0 Axes>

In [10]:
splt.oneitem_iterate_errorbar(
    len( grouped_data ),
    lambda d: grouped_data[d]['dep_polymers'],
    lambda d: grouped_data[d]['defects_frac'],
    lambda d: grouped_data[d]['gamma'],
    lambda d: [
        np.max( [ grouped_data[d]['B'], ufloat( 1e-9, 1e-10 ) ] )
    ],
    lambda it, l: [
        { 'label': fr"$q = {l}$", 'ls': '-', 'marker': 's' },
    ],
    lambda it: (
        plt.title( fr"{it}" ),
        plt.xlabel( fr"$\gamma$" ),
        plt.ylabel( fr"$B$" ),
        plt.yscale( 'log' ),
        plt.ylim( 1e-10, 1e-3 ),
        plt.yticks(
            [ 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9 ],
            [ r"$10^{-4}$", r"$10^{-5}$", r"$10^{-6}$", r"$10^{-7}$", r"$10^{-8}$", r"$\approx$ 0" ] ),
            plt.minorticks_off()
        # splt.hline( 0 )
    )
)

plt.tight_layout()
plt.savefig( savedir + '/B.pdf' )
# plt.savefig( savedir + '/B.png' )

<Figure size 432x0 with 0 Axes>